In [3]:
# load autoload plugins for jupyter notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import torch
from easydict import EasyDict as edict
from dataset_util import NusceneDataset
from voxelization import Voxelization
from voxelnet import VoxelNet

torch.set_default_device("cuda:0")
torch.set_default_dtype(torch.float32)

cfg = edict()
cfg.x_range = [-50.4,50.4]
cfg.y_range = [-40.,40.]
cfg.z_range = [-1.,3.]
cfg.voxel_size = {'x':0.2, 'y':0.2, 'z':0.4}
cfg.max_voxel_pts = 35
cfg.max_points_per_cloud = 25000
cfg.pos_neg_iou_thresh = [0.6,0.45]

def get_canvas_size(rng, res):
    return int(np.round((rng[1]-rng[0])/res))

cfg.canvas_size = {"h":get_canvas_size(cfg.y_range, 2*cfg.voxel_size["y"]),
                   "w":get_canvas_size(cfg.x_range, 2*cfg.voxel_size["x"])}
cfg.canvas_res = {"w":2*cfg.voxel_size["x"], "h":2*cfg.voxel_size["y"]}
cfg.canvas_min = {"h":cfg.y_range[0], "w":cfg.x_range[0]}

cfg.anchors = [{"z":0.,"l":3.9,"w":1.6,"h":1.56,"yaw":0.0},
               {"z":0.,"l":3.9,"w":1.6,"h":1.56,"yaw":np.pi/2.0},]


cfg.voxelizer = Voxelization(x_range=cfg.x_range,y_range=cfg.y_range,
                        z_range=cfg.z_range,
                        voxel_size=cfg.voxel_size,
                        max_voxel_pts=cfg.max_voxel_pts,
                        init_decoration=True)

nd_train = NusceneDataset(path="/media/ji/Samsung_T5/nuScene_mini/v1.0-mini",
                    version="v1.0-mini",
                    canvas_h=cfg.canvas_size["h"],
                    canvas_w=cfg.canvas_size["w"],
                    canvas_res=cfg.canvas_res,
                    canvas_min=cfg.canvas_min,
                    anchors=cfg.anchors,
                    voxelizer=cfg.voxelizer,
                    max_pts_per_cloud=cfg.max_points_per_cloud,
                    pos_neg_iou_thresh=cfg.pos_neg_iou_thresh,
                    mode="train")

Loading NuScenes tables for version v1.0-mini...


2024-07-12 17:29:56.340 | INFO     | dataset_util:__init__:400 - Total number of samples: 404
2024-07-12 17:29:56.341 | INFO     | dataset_util:__init__:413 - Number of samples for train: 323


23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.473 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [8]:
nd_train.sample_tokens[:10]

['639a4120779f44bb9cf41be3e8612e36',
 '8bf071b1b92c4ff1bd5016bfb30de3f7',
 'ace4b20b94cf47d0b68addac79fe68de',
 'e8b1863300964b2481fee312496d06d8',
 '82597244941b4e79aa3e1e9cc6386f8b',
 '4cf5d6c3f6ab42aab23f67b5a9782d1a',
 '3f2ddc9476eb475b8c46f777f508cedb',
 '9cdbf5ff7f294549aea0a4307e5d104a',
 '5b03af7a953245b5a3b23191ed4da62a',
 'cc57c1ea80fe46a7abddfdb15654c872']

In [9]:
nd_test = NusceneDataset(path="/media/ji/Samsung_T5/nuScene_mini/v1.0-mini",
                    version="v1.0-mini",
                    canvas_h=cfg.canvas_size["h"],
                    canvas_w=cfg.canvas_size["w"],
                    canvas_res=cfg.canvas_res,
                    canvas_min=cfg.canvas_min,
                    anchors=cfg.anchors,
                    voxelizer=cfg.voxelizer,
                    max_pts_per_cloud=cfg.max_points_per_cloud,
                    pos_neg_iou_thresh=cfg.pos_neg_iou_thresh,
                    mode="test")

nd_test.sample_tokens[:10]

Loading NuScenes tables for version v1.0-mini...


2024-07-12 17:31:08.246 | INFO     | dataset_util:__init__:400 - Total number of samples: 404
2024-07-12 17:31:08.247 | INFO     | dataset_util:__init__:413 - Number of samples for test: 81


23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.524 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


['cc0342fe162348dc9445493aab4b463e',
 'f90292e4cdf647e1a8842baabdaafcec',
 'cb4e6195faad467094fbd4d0a9e960e9',
 '8092909473464f80b9f791a4d31ddcb8',
 'fe42b85d7f7a4107b650ffa993a4fbff',
 'a34fabc7aa674713b71f98ec541eb2d4',
 '4786b3c0d0a24c8b86b0676d32dd5919',
 'dde297c8e12f44c4bc2f2c80e49d2b5a',
 'be1494c7762c43ca88010f366997a586',
 '17a9388ec4b04e028b4d07af2cc3a45c']

In [10]:
nd_train.canvas_h, nd_train.canvas_w

(200, 252)

In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(nd_train, batch_size=2,shuffle=False,
                              generator=torch.Generator(device='cuda'))
test_dataloader = DataLoader(nd_test, batch_size=1,shuffle=True,
                             generator=torch.Generator(device='cuda'))

In [14]:
input = next(iter(train_dataloader))

n_vox: 8473


In [6]:
vnet = VoxelNet(x_range=cfg.x_range, y_range=cfg.y_range, z_range=cfg.z_range,
                voxel_size=cfg.voxel_size, anchors=cfg.anchors,
                max_voxel_pts=cfg.max_voxel_points)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
vnet.to(device)

cuda


VoxelNet(
  (voxelization): Voxelization()
  (voxel_feature): VoxelFeatureExtraction()
  (scn): SpatialConvolution(
    (net): SparseSequential(
      (0): SparseConv3d(128, 64, kernel_size=[3, 3, 3], stride=[2, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.Native)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): SparseConv3d(64, 64, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[0, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.Native)
      (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): SparseConv3d(64, 64, kernel_size=[3, 3, 3], stride=[2, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], algo=ConvAlgo.Native)
      (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
  )
  (rpn): RegionProposalNet(
    (bloc

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(nd_train, batch_size=1, shuffle=True,generator=torch.Generator(device='cuda'))
test_dataloader = DataLoader(nd_test, batch_size=1, shuffle=True,generator=torch.Generator(device='cuda'))

In [8]:
import time
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from loguru import logger

# training loop


In [10]:

def train(n_epoch, vnet, train_dataloader, test_dataloader):

    vnet.train()

    optimizer = optim.Adam(vnet.parameters(), lr=0.001)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                               factor=0.1, patience=5,
                                               threshold=1e-2,
                                               min_lr=1e-6)
    
    for epoch in range(n_epoch):

        for ibatch, input in enumerate(train_dataloader):
            optimizer.zero_grad()
            t0 = time.time_ns()
            mdl_output, loss = vnet(input)
            loss.backward()
            t1 = time.time_ns()
            optimizer.step()

            # print loss
            if ibatch % 1 == 0:
                logger.info(
                    f"Epoch: {epoch}, Batch: {ibatch}, Loss: {loss.item()}")
                logger.info(f"Per batch time: {(t1-t0)/1e9} sec")
        
        scheduler.step(loss)


In [11]:
train(10, vnet, train_dataloader, test_dataloader)

torch.Size([1, 2, 200, 252]) torch.Size([1, 14, 200, 252])


2024-07-11 16:44:11.951 | INFO     | __main__:train:23 - Epoch: 0, Batch: 0, Loss: 4.748961448669434
2024-07-11 16:44:11.952 | INFO     | __main__:train:25 - Per batch time: 1.95570089 sec
2024-07-11 16:44:13.183 | INFO     | __main__:train:23 - Epoch: 0, Batch: 1, Loss: 4.164446830749512
2024-07-11 16:44:13.184 | INFO     | __main__:train:25 - Per batch time: 1.14365374 sec


torch.Size([1, 2, 200, 252]) torch.Size([1, 14, 200, 252])


2024-07-11 16:44:14.629 | INFO     | __main__:train:23 - Epoch: 0, Batch: 2, Loss: 4.117429256439209
2024-07-11 16:44:14.630 | INFO     | __main__:train:25 - Per batch time: 1.260314471 sec


torch.Size([1, 2, 200, 252]) torch.Size([1, 14, 200, 252])


...

2024-07-11 16:53:52.590 | INFO     | __main__:train:23 - Epoch: 1, Batch: 120, Loss: 2.3642778396606445
2024-07-11 16:53:52.591 | INFO     | __main__:train:25 - Per batch time: 1.343707152 sec


torch.Size([1, 2, 200, 252]) torch.Size([1, 14, 200, 252])


2024-07-11 16:53:54.067 | INFO     | __main__:train:23 - Epoch: 1, Batch: 121, Loss: 2.534032106399536
2024-07-11 16:53:54.068 | INFO     | __main__:train:25 - Per batch time: 1.265817224 sec


torch.Size([1, 2, 200, 252]) torch.Size([1, 14, 200, 252])


KeyboardInterrupt: 